## ECOPOTENTIAL-00003 Data Pipeline Coodinator Queue - NDVI Donana

This coordinator feeds the ecopotential-00003 data pipeline queue for the Donana protected area

### Data pipeline parameters

In [1]:
data_pipeline = 'ecopotential-00003'

In [2]:
wps_url_triggers = 'https://ec-ecopotential-triggers-deployer.terradue.com/zoo/'
wps_url_apps = 'https://ec-ecopotential-apps-deployer.terradue.com/zoo/'

### Data transformation application parameters

* Application: dcs-s2-ndvi-boa
* Application repository: https://github.com/ec-ecopotential/dcs-s2-ndvi-boa
* Build (CI): https://build.terradue.com/job/communities/job/Github%20Organizations/job/ec-ecopotential/job/dcs-s2-ndviboa/

In [3]:
process_id = 'ec_ecopotential_dcs_s2_ndvi_boa_ndvi_boa_1_9'

In [4]:
pa_code = 'DO'
pa_name = 'Donana'
resolution = '10'
plot = 'False'
flag_expr = 'saturated_l1a_B4'

### Data selection parameters

In [5]:
series = 'https://catalog.terradue.com/sentinel2/description'

In [6]:
product_type = 'S2MSI2A'

In [7]:
cloud_coverage = '100['

In [8]:
wkt = 'POLYGON((-6.79440295589744 37.1191319871777,-6.80827154725944 37.1673031942056,-6.72121114356795 37.2425767108835,-6.71042775152577 37.2930498879682,-6.73405462341917 37.3301447063348,-6.71685147187095 37.3522253716156,-6.67435624435345 37.3534492963332,-6.64461873043217 37.3365633914571,-6.62625336367183 37.3569789562915,-6.59581018720718 37.3506994424118,-6.58125246058411 37.3703051383248,-6.492413660497 37.3590379521576,-6.4716624422663 37.3807359777278,-6.43859705183134 37.368908852933,-6.44398095303713 37.347088527492,-6.42200724935979 37.3329929700694,-6.37035012694059 37.3583637388823,-6.33153540861988 37.3329986280188,-6.25703879672152 37.3486795386779,-6.2123425985781 37.3266608721382,-6.1976208676784 37.283024780795,-6.17366048732846 37.2660742144794,-6.15177710683581 37.2667431439207,-6.14475916204647 37.3055642119627,-6.1153577846492 37.3105672777454,-6.04475036124726 37.2771976083931,-6.04063041298302 37.2060457158668,-6.01561413928283 37.1744528799079,-6.03401959555254 37.1582677514823,-6.01528760169561 37.1211642619874,-6.03587498721126 37.0508366627984,-6.08415258431292 37.0557470251154,-6.1443715598251 37.0182321851901,-6.20489081698833 36.9013662293736,-6.24848183495607 36.8846441975983,-6.28133674315331 36.9000390906331,-6.27285487885334 36.8641095189495,-6.21240564569216 36.8569441225281,-6.19273049982008 36.8273222937041,-6.23650318117893 36.7815132843624,-6.25972339425557 36.7794168824025,-6.29452828553456 36.7134050019461,-6.32211152509425 36.693679078589,-6.40354402587554 36.7535989999008,-6.39366914786855 36.7750231637474,-6.42465019950879 36.7928589511795,-6.48913510482893 36.9063367348853,-6.55574132052792 36.9640015939793,-6.54729285159512 36.9812830814621,-6.79440295589744 37.1191319871777))'

In [9]:
from shapely.wkt import loads 

In [10]:
geom = loads(wkt).buffer(0.2).simplify(0.01, preserve_topology=False).to_wkt().replace(',', '%2C').replace(' ', '%20')

### User Mapping

In [11]:
api_key = 'AKCp5aTSAokLQTzhVH52Q8tcfobtaVQtJ3xPiGgB8AdxSPvDpUXU1DW2uYby4EjJcEiJtaoPu'

### Coordinator parameters

In [12]:
start_queue = '${coord:formatTime(coord:dateOffset(coord:nominalTime(), -10, \'DAY\'), "yyyy-MM-dd\'T\'HH:mm:ss\'Z\'")}'
end_queue = '${coord:formatTime(coord:dateOffset(coord:nominalTime(), -9, \'DAY\'), "yyyy-MM-dd\'T\'HH:mm:ss\'Z\'")}'

In [13]:
co_trigger_queue_process_id = 'coordinator_ec_ecopotential_tg_ecopotential_00003_queue_tg_ecopotential_00003_1_12'

In [14]:
coordinator_date_start = '2018-04-01T00:00Z'
coordinator_date_stop = '2018-12-31T00:00Z'
coordinator_period = '0 0 * * *'

### Imports

In [15]:
import owslib
from owslib.wps import monitorExecution
from owslib.wps import WebProcessingService
import sys
import os
sys.path.append('/opt/anaconda/bin/')
import lxml.etree as etree
import requests
import shapely
import dateutil.parser

from datetime import datetime, timedelta
from io import BytesIO
import json

### Common Parameters

In [16]:
tg_quotation = 'No'
recovery = 'No'
_T2Username = data_pipeline

### Check data transformation application

In [17]:
wps = WebProcessingService(wps_url_apps, verbose=False, skip_caps=False)

In [18]:
found_process = False

message = "The process %s is not deployed" % process_id  
    
for index, elem in enumerate(wps.processes):

    if elem.identifier == process_id:
        message = "The process %s is deployed" % process_id  
        found_process = True
        
print message

if not found_process:
    raise Exception()

The process ec_ecopotential_dcs_s2_ndvi_boa_ndvi_boa_1_9 is deployed


In [19]:
process = wps.describeprocess(process_id)

In [20]:
for input in process.dataInputs:
    print(input.identifier)

source
pa_code
pa_name
resolution
plot
flag_expr
_T2Username


### Check trigger coordinator

In [21]:
wps = WebProcessingService(wps_url_triggers, verbose=False, skip_caps=False)

In [22]:
found_process = False

message = "The queue coordinator process %s is not deployed" % co_trigger_queue_process_id  
    
for index, elem in enumerate(wps.processes):

    if elem.identifier == co_trigger_queue_process_id:
        message = "The queue coordinator process %s is deployed" % co_trigger_queue_process_id  
        found_process = True
        
print message

if not found_process:
    raise Exception()

The queue coordinator process coordinator_ec_ecopotential_tg_ecopotential_00003_queue_tg_ecopotential_00003_1_12 is deployed


### Feed the queue

In [23]:
process = wps.describeprocess(co_trigger_queue_process_id)

In [24]:
process.title

'Trigger for ecopotential-00003 Coordinator'

In [25]:
process.abstract

'Coordinator: Trigger for ecopotential-00003'

In [26]:
for input in process.dataInputs:
    print(input.identifier)

series
data_pipeline
wps_url
process_id
update
geom
product_type
cloud_coverage
tg_quotation
api_key
recovery
mode
pa_code
pa_name
resolution
plot
flag_expr
t2_coordinator_date_start
t2_coordinator_date_stop
t2_coordinator_period
_T2Username


##### Define the input parameters


In [27]:
mode = 'Queue'

In [28]:
inputs = [('series', series),
        ('data_pipeline', data_pipeline),
        ('wps_url', wps_url_apps),
        ('process_id', process_id),
        ('update', '%s/%s' % (start_queue, end_queue)),
        ('geom', geom),
        ('product_type', product_type),
        ('cloud_coverage', cloud_coverage),
        ('tg_quotation', tg_quotation),
        ('api_key', api_key),
        ('recovery', recovery),
        ('mode', mode),
        ('pa_code', pa_code),
        ('pa_name', pa_name),
        ('resolution', resolution),
        ('plot', plot),
        ('flag_expr', flag_expr),
        ('t2_coordinator_date_start', coordinator_date_start), 
        ('t2_coordinator_date_stop', coordinator_date_stop),
        ('t2_coordinator_period', coordinator_period),
        ('_T2Username', data_pipeline)]

##### Identify the outputs

In [29]:
for output in process.processOutputs:
    print(output.identifier)

coordinatorIds


##### Submit the coordinator request

In [30]:
execution = owslib.wps.WPSExecution(url=wps.url)

In [31]:
execution_request = execution.buildRequest(co_trigger_queue_process_id, 
                                           inputs, 
                                           output = [('coordinatorIds', False)])

In [32]:
execution_response = execution.submitRequest(etree.tostring(execution_request, pretty_print=True))

In [33]:
if False:
    print(etree.tostring(execution_request, pretty_print=True))

In [34]:
execution.parseResponse(execution_response)

In [35]:
if False:
    print(etree.tostring(execution_response, pretty_print=True))

In [36]:
execution.statusLocation

'http://ec-ecopotential-triggers-deployer.terradue.com/zoo-bin/zoo_loader.cgi?request=Execute&service=WPS&version=1.0.0&Identifier=GetStatus&DataInputs=sid=9a06775c-6e29-11e8-9974-0242ac110005&RawDataOutput=Result'

In [38]:
monitorExecution(execution)

In [39]:
execution.isSucceded()

True

In [40]:
for output in execution.processOutputs:
    print(output.identifier)

coordinatorIds
result_osd
QUOTATION


In [41]:
coordinator_id = str(json.loads(execution.processOutputs[0].data[0])['coordinatorsId'][0]['oozieId'])

In [42]:
coordinator_id

'0002157-180330140554685-oozie-oozi-C'